# PostProcessing of the whole lane

In [57]:
import cv2
from pathlib import Path
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas as pd
from scipy.signal import savgol_filter
from scipy.signal import medfilt
from scipy.spatial import procrustes
from scipy.optimize import minimize

Import video

In [58]:
video_number = "3"
# Define the relative path to the video file
notebook_dir = Path().resolve()
project_root = notebook_dir.parent.parent
print(f"Project root: {project_root}")
video_path = project_root / "data" / f"recording_{video_number}" / f"Recording_{video_number}.mp4" 
video_path = str(video_path)

# Load the video
cap = cv2.VideoCapture(video_path)

# Check
print(f"Opened: {cap.isOpened()}, FPS: {cap.get(cv2.CAP_PROP_FPS)}, Total Frames: {cap.get(cv2.CAP_PROP_FRAME_COUNT)}")

Project root: C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis
Opened: True, FPS: 59.94005994005994, Total Frames: 227.0


Import lane points

In [59]:
# Define the path to the CSV file
input_data_path = project_root / "data" / "auxiliary_data" / "lane_points" / f"lane_points_raw_{video_number}.csv" # may change raw to processed

# Load the CSV file into a DataFrame
points_df = pd.read_csv(input_data_path)
# Display the first few rows of the DataFrame
# print(points_df.head())

Draw the lines on a frame

In [60]:
'''Disegna la linea sul frame'''
def write_line_on_frame(frame, line):
    # Create a copy of the original frame to draw the first line
    modified_frame = np.copy(frame)

    # Extract the first line's rho and theta
    if line is not None:
        x1, y1, x2, y2 = line
        
        # Draw the first line on the frame
        cv2.line(modified_frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

    # return the modified frame
    return modified_frame

In [61]:
def write_lines_on_frame(frame, lines):
    for i in range(len(lines)):
        # print('linea', i, ':', lines[i])
        frame = write_line_on_frame(frame, lines[i])
    return frame

From the 4 points get the 4 linies to draw

In [62]:
def lines_from_points(df):
    lines =[]
    for i in range(len(df)):
        bottom = [df.iloc[i]['bottom_right_x'], df.iloc[i]['bottom_right_y'], df.iloc[i]['bottom_left_x'], df.iloc[i]['bottom_left_y']]
        top = [df.iloc[i]['up_right_x'], df.iloc[i]['up_right_y'], df.iloc[i]['up_left_x'], df.iloc[i]['up_left_y']]
        left = [df.iloc[i]['bottom_left_x'], df.iloc[i]['bottom_left_y'], df.iloc[i]['up_left_x'], df.iloc[i]['up_left_y']]
        right = [df.iloc[i]['bottom_right_x'], df.iloc[i]['bottom_right_y'], df.iloc[i]['up_right_x'], df.iloc[i]['up_right_y']]
        
        # Append the line to the list
        lines.append([bottom, top, left, right])
    return lines

## Sav-Gol filter

In [63]:
# Create a new 8D array from your points
quad_points = points_df[[
    'bottom_left_x', 'bottom_left_y',
    'bottom_right_x', 'bottom_right_y',
    'up_left_x', 'up_left_y',
    'up_right_x', 'up_right_y'
]].values

med_points = medfilt(quad_points, kernel_size=(1, 1))

# Apply savgol filter across the entire matrix (axis=0 means apply filter column-wise)
smoothed_quad = savgol_filter(quad_points, window_length=5, polyorder=1, axis=0)

smoothed_quad = smoothed_quad.astype(int)

# Update DataFrame
df_smoothed = points_df.copy()
df_smoothed.loc[:, [
    'bottom_left_x', 'bottom_left_y',
    'bottom_right_x', 'bottom_right_y',
    'up_left_x', 'up_left_y',
    'up_right_x', 'up_right_y'
]] = med_points


## Procrustes Analysis

In [64]:
''' Align each frame quadrilateral to the mean quadrilateral using Procrustes analysis
    It allpies a similarity transformation to minimize the disparity between two shapes. (rotation, translation, scaling)
'''

# Define the columns for the quadrilateral points

point_cols = [
    'bottom_left_x', 'bottom_left_y',
    'bottom_right_x', 'bottom_right_y',
    'up_left_x', 'up_left_y',
    'up_right_x', 'up_right_y']

# Extract quadrilateral as array (N, 4, 2)
def df_to_quads(points_df):
    return points_df[point_cols].values.reshape((-1, 4, 2))

quads = df_to_quads(points_df)

# Choose reference shape: e.g., the first frame, or average shape
reference_shape = np.mean(quads, axis=0)  # mean quadrilateral

print("Reference shape:", reference_shape)

# Align each frame's quad to reference using Procrustes
aligned_quads = []

# with procrustes
for quad in quads: 
    # Procrustes expects same shape and centered/normalized input
    mtx1, mtx2, disparity = procrustes(reference_shape, quad)
    aligned_quads.append(mtx1)  # you can use mtx1 to get the ideal shape, mtx2 is the aligned version



aligned_quads = np.array(aligned_quads)  # shape (N, 4, 2)
print(aligned_quads[0])
# Convert back to DataFrame
flattened = aligned_quads.reshape((aligned_quads.shape[0], -1))
print(flattened[0])
smoothed_df = points_df.copy()
smoothed_df[point_cols] = flattened

df_procrustes = smoothed_df.copy()

Reference shape: [[ 454.85462555 1036.78414097]
 [1126.33039648 1049.10132159]
 [1106.17180617  402.40528634]
 [1367.17621145  404.03964758]]
[[-0.59982921  0.33674757]
 [ 0.12097641  0.34996963]
 [ 0.09933687 -0.34423582]
 [ 0.37951592 -0.34248139]]
[-0.59982921  0.33674757  0.12097641  0.34996963  0.09933687 -0.34423582
  0.37951592 -0.34248139]


Similarity

In [65]:
def compute_quadrilater_center(quad):
    return np.mean(quad, axis=0)

In [66]:

''' Align each frame quadrilateral to the mean quadrilateral using similarity transform
    It allpies a similarity transformation to minimize the disparity between two shapes. (rotation, translation, scaling)
    Then it align the quadrilateral with one chosen reference point
    It solves the problem of bottom line not being in the frame
    Not very good for points processing, it still jitters
'''

# Define the columns for the quadrilateral points

point_cols = [
    'bottom_left_x', 'bottom_left_y',
    'bottom_right_x', 'bottom_right_y',
    'up_left_x', 'up_left_y',
    'up_right_x', 'up_right_y']

# Extract quadrilateral as array (N, 4, 2)
def df_to_quads(points_df):
    return points_df[point_cols].values.reshape((-1, 4, 2))

quads = df_to_quads(points_df)

# Choose reference shape: e.g., the first frame, or average shape
reference_shape = np.mean(quads, axis=0)  # mean quadrilateral


print("Reference shape:", reference_shape)

# Align each frame's quad to reference using Procrustes
aligned_quads = []


# with similarity transform
i = 0
for quad in quads:
    # Estimate similarity transform (rotation + translation + uniform scale)
    transform_matrix, _ = cv2.estimateAffinePartial2D(quad, reference_shape, method=cv2.LMEDS)

    # Apply transform to quad
    aligned = cv2.transform(np.array([quad]), transform_matrix)[0]

    # Shift the quadrilateral in a way that the reference point is aligned to the original reference point int the original image
    reference_point_new =compute_quadrilater_center(aligned)
    reference_point_old = compute_quadrilater_center(quad)

    shift = [reference_point_old[0] - reference_point_new[0], reference_point_old[1] - reference_point_new[1]]

    # Apply shift to entire aligned quadrilateral
    shifted_quad = aligned + shift

    aligned_quads.append(shifted_quad)

    i += 1

aligned_quads = np.array(aligned_quads)  # shape (N, 4, 2)
print(aligned_quads[0])
# Convert back to DataFrame
flattened = aligned_quads.reshape((aligned_quads.shape[0], -1))
print(flattened[0])
smoothed_df = points_df.copy()
smoothed_df[point_cols] = flattened

df_similarity = smoothed_df.copy()



Reference shape: [[ 454.85462555 1036.78414097]
 [1126.33039648 1049.10132159]
 [1106.17180617  402.40528634]
 [1367.17621145  404.03964758]]
[[ 608.25 1048.25]
 [1323.25 1058.25]
 [1244.25  436.25]
 [1548.25  439.25]]
[ 608.25 1048.25 1323.25 1058.25 1244.25  436.25 1548.25  439.25]


## With Optimization 
Preserve Lane Lengths in consecutive frames

In [67]:
''' It works well if the first frame is correct'''

def edge_lengths(quad):
    return np.linalg.norm(np.roll(quad, -1, axis=0) - quad, axis=1)

def cost_function(x, prev_lengths):
    # x: flattened coords of 4 points relative to center
    rel_quad = x.reshape(4, 2)
    lengths = edge_lengths(rel_quad)
    return np.sum((lengths - prev_lengths) ** 2)

# Initial data: quads (original), shape (N, 4, 2)
centers = np.mean(quads, axis=1)  # shape (N, 2)
smoothed_quads = []

# First frame: use as is
smoothed_quads.append(quads[0])
prev_quad = quads[0]

for t in range(1, len(quads)):
    center = centers[t]  # fixed center for this frame
    prev_lengths = edge_lengths(prev_quad)

    # Use previous relative positions from center as initial guess
    rel_prev = prev_quad - compute_quadrilater_center(prev_quad)
    x0 = rel_prev.flatten()

    # Optimize shape while preserving center
    res = minimize(cost_function, x0, args=(prev_lengths,), method='L-BFGS-B')
    rel_quad = res.x.reshape(4, 2)
    new_quad = rel_quad + center  # add back center

    smoothed_quads.append(new_quad)
    prev_quad = new_quad

# Convert to array
smoothed_quads = np.array(smoothed_quads)

# Update DataFrame
flattened = smoothed_quads.reshape((smoothed_quads.shape[0], -1))
df_smoothed_center = points_df.copy()
df_smoothed_center[point_cols] = flattened


Preserve lane lengths with respect the average one

In [68]:
from scipy.optimize import minimize
import numpy as np

def quad_center(quad):
    return np.mean(quad, axis=0)

# Step 1: Compute relative positions for all quads
rel_quads = quads - np.mean(quads, axis=1, keepdims=True)  # (N, 4, 2)

# Step 2: Compute the average shape (relative to center)
average_rel_shape = np.mean(rel_quads, axis=0)  # (4, 2)

# Step 3: Optimization function to minimize deviation from avg shape
def cost_to_avg_shape(x, avg_shape):
    rel_quad = x.reshape(4, 2)
    return np.sum((rel_quad - avg_shape)**2)

# Step 4: Optimize each frame
smoothed_quads = []

for t in range(len(quads)):
    center = quad_center(quads[t])

    # Initial guess: current relative shape
    rel_init = quads[t] - center
    x0 = rel_init.flatten()

    # Minimize distance from average shape
    res = minimize(cost_to_avg_shape, x0, args=(average_rel_shape,), method='L-BFGS-B')
    
    # Reconstruct quad
    rel_quad = res.x.reshape(4, 2)
    new_quad = rel_quad + center
    smoothed_quads.append(new_quad)

# Convert result
smoothed_quads = np.array(smoothed_quads)

# Update DataFrame
flattened = smoothed_quads.reshape((smoothed_quads.shape[0], -1))
df_smoothed_to_avg_shape = points_df.copy()
df_smoothed_to_avg_shape[point_cols] = flattened


Apply the average quadrilater in the center of each detection

In [69]:
# compute the average shape
average_shape = np.mean(quads, axis=0)  # shape (4, 2)

# compute the center of the average shape
average_shape_center = np.mean(average_shape, axis=0)
# compute the relative positions of the average shape points to its center
relative_average_shape = average_shape - average_shape_center

# in each frame compute the center of the quadrilateral and apply to it the relative average shape
new_quads = []
for t in range(len(quads)):
    center = quad_center(quads[t])
    new_quad = relative_average_shape + center
    new_quads.append(new_quad)

# Convert result
new_quads = np.array(new_quads)
# Update DataFrame
flattened = new_quads.reshape((new_quads.shape[0], -1))
df_avg_shape = points_df.copy()
df_avg_shape[point_cols] = flattened

# display the first few rows of the DataFrame
# print(df_avg_shape.head())
# print(points_df.head())

# print the average of each row of df_avg_shape and points_df
# print(df_avg_shape.mean(axis=1))
# print(points_df.mean(axis=1))

Postprocessing on the points

In [70]:
def Savitzky_Golay_filter(points, window_length=25, polyorder=3):
    """
    Smooths the X and Y coordinates using Savitzky-Golay filter to reduce noise.
    """

    points[:,0] = savgol_filter(points[:,0], window_length=window_length, polyorder=polyorder)
    points[:,1] = savgol_filter(points[:,1], window_length=window_length, polyorder=polyorder)

    
    return points

In [71]:
def process_point(points):
    Savitzky_Golay_filter(points, window_length=5, polyorder=1) 
    # points = medfilt(points, kernel_size=(3, 1))
    return points

In [72]:
bottom_left = points_df[['bottom_left_x', 'bottom_left_y']].values
bottom_right = points_df[['bottom_right_x', 'bottom_right_y']].values
up_left = points_df[['up_left_x', 'up_left_y']].values
up_right = points_df[['up_right_x', 'up_right_y']].values

bottom_left_processed = process_point(bottom_left)
bottom_right_processed = process_point(bottom_right)
up_left_processed = process_point(up_left)
up_right_processed = process_point(up_right)


# Create a new DataFrame with the processed points
processed_df = pd.DataFrame({
    'bottom_left_x': bottom_left_processed[:, 0],
    'bottom_left_y': bottom_left_processed[:, 1],
    'bottom_right_x': bottom_right_processed[:, 0],
    'bottom_right_y': bottom_right_processed[:, 1],
    'up_left_x': up_left_processed[:, 0],
    'up_left_y': up_left_processed[:, 1],
    'up_right_x': up_right_processed[:, 0],
    'up_right_y': up_right_processed[:, 1]
})

'''USELESS'''

'USELESS'

PostProcessing on the distances between points

In [73]:
left_line = points_df[['bottom_left_x', 'bottom_left_y', 'up_left_x', 'up_left_y']].values
right_line = points_df[['bottom_right_x', 'bottom_right_y', 'up_right_x', 'up_right_y']].values
bottom_line = points_df[['bottom_left_x', 'bottom_left_y', 'bottom_right_x', 'bottom_right_y']].values
up_line = points_df[['up_left_x', 'up_left_y', 'up_right_x', 'up_right_y']].values
# compute the difference between the two endpoints of the line
left_line_diff = left_line[:, 2:] - left_line[:, :2]
right_line_diff = right_line[:, 2:] - right_line[:, :2]
bottom_line_diff = bottom_line[:, 2:] - bottom_line[:, :2]
up_line_diff = up_line[:, 2:] - up_line[:, :2]

left_processed_diff = process_point(left_line_diff)
right_processed_diff = process_point(right_line_diff)
bottom_processed_diff = process_point(bottom_line_diff)
up_processed_diff = process_point(up_line_diff)

left_processed = left_processed_diff + left_line[:, :2]
right_processed = right_processed_diff + right_line[:, :2]
bottom_processed = bottom_processed_diff + bottom_line[:, :2]
up_processed = up_processed_diff + up_line[:, :2]

# Create a new DataFrame with the processed points
# processed_df = pd.DataFrame({
#     'bottom_left_x': bottom_line[:, 0],
#     'bottom_left_y': bottom_line[:, 1],
#     'bottom_right_x': bottom_processed[:, 0],
#     'bottom_right_y': bottom_processed[:, 1],
#     'up_left_x': up_line[:, 0],
#     'up_left_y': up_line[:, 1],
#     'up_right_x': up_processed[:, 0],
#     'up_right_y': up_processed[:, 1]
# })

processed_df = pd.DataFrame({
    'bottom_left_x': left_processed[:, 0],
    'bottom_left_y': left_processed[:, 1],
    'bottom_right_x': right_processed[:, 0],
    'bottom_right_y': right_processed[:, 1],
    'up_left_x': left_line[:, 0],
    'up_left_y': left_line[:, 1],
    'up_right_x': right_line[:, 0],
    'up_right_y': right_line[:, 1]
})

##  Modify the video

In [74]:
# Reset the video to the beginning
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

# Define the codec and create a VideoWriter object to save the modified frames
output_path = project_root / "data" / f"recording_{video_number}"  / "Lane_detection.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Use 'mp4v' codec for MP4 format
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))

# Process the lines
lines = lines_from_points(points_df)
# print('Processed lines:', lines)


# Loop through each frame in the video
frame_index = 0
while frame_index < len(lines):
    ret, video_frame = cap.read()
    if not ret:
        print("End of video or failed to read the frame at iteration", frame_index)
        break
    # print(f"Processing frame {frame_index}")

    # draw the lines on the frame   
    modified_frame = write_lines_on_frame(video_frame, lines[frame_index])


    # Write the modified frame to the output video
    out.write(modified_frame)

    # Increment the frame index
    frame_index += 1

# Release the video capture and writer objects
# cap.release()
out.release()

print(f"Adjusted video saved to {output_path}")

Adjusted video saved to C:\Users\miche\OneDrive\Documenti\GitHub\bowling-analysis\data\recording_3\Lane_detection.mp4
